In [112]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import lightgbm as lgb
import optuna
import streamlit as st
import plotly
import joblib

print("ALHAMDULILLAH! Semua library berhasil di-load.")
print("-" * 30)
print(f"Pandas version: {pd.__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"Streamlit version: {st.__version__}")

ALHAMDULILLAH! Semua library berhasil di-load.
------------------------------
Pandas version: 2.3.3
LightGBM version: 4.6.0
Scikit-learn version: 1.8.0
Optuna version: 4.6.0
Streamlit version: 1.52.1


In [113]:
import os

file_path = r'E:\airta drafts\PREDIKSI KADAR HB\data\raw\erm_hd.xlsx'

try:
    df = pd.read_excel(file_path)
    display(df.head())

    print("\n Informasi Dataset:")
    df.info()

except Exception as e:
    print(f"Terjadi kesalahan: {e}")

,id_pasien,jenis_kelamin,tgl_lahir,tgl_pemeriksaan,eritrosit,hematokrit,MCHC,MCH,MCV,hemoglobin,leukosit,trombosit,status_epo
0,1,P,1965-07-04,2025-03-01,2.8,25.4,33.9,30.2,89.4,8.6,7830,309000,TIDAK
1,1,P,1965-07-04,2025-04-02,2.7,24.3,33.3,29.7,88.9,8.1,7150,262000,TIDAK
2,1,P,1965-07-04,2025-05-03,2.8,24.5,33.9,29.8,88.4,8.3,8070,297000,TIDAK
3,1,P,1965-07-04,2025-06-04,2.7,24.0,32.9,29.2,88.6,7.9,7960,291000,TIDAK
4,1,P,1965-07-04,2025-07-02,2.5,22.5,33.3,29.7,89.5,7.5,7860,307000,TIDAK



 Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_pasien        829 non-null    int64  
 1   jenis_kelamin    829 non-null    object 
 2   tgl_lahir        829 non-null    object 
 3   tgl_pemeriksaan  829 non-null    object 
 4   eritrosit        829 non-null    object 
 5   hematokrit       829 non-null    object 
 6   MCHC             829 non-null    object 
 7   MCH              829 non-null    object 
 8   MCV              829 non-null    object 
 9   hemoglobin       829 non-null    float64
 10  leukosit         828 non-null    object 
 11  trombosit        828 non-null    object 
 12  status_epo       829 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 84.3+ KB


In [114]:
# Inisialisasi df_clean dari df asal jika belum
df_clean = df.copy()

# ==========================================
# LANGKAH 1: KONVERSI DATATYPE
# ==========================================
cols_numeric = ['eritrosit', 'hematokrit', 'MCHC', 'MCH', 'MCV', 'hemoglobin', 'leukosit', 'trombosit']

for col in cols_numeric:
    # Menghapus spasi dan mengubah ke numeric (mengatasi tanda titik/koma)
    df_clean[col] = pd.to_numeric(df_clean[col].astype(str).str.strip().str.replace(',', '.'), errors='coerce')

# ==========================================
# LANGKAH 2: KONVERSI TGL_LAHIR KE USIA (INTEGER)
# ==========================================
# Pastikan kedua kolom bertipe datetime
df_clean['tgl_lahir'] = pd.to_datetime(df_clean['tgl_lahir'], errors='coerce')
df_clean['tgl_pemeriksaan'] = pd.to_datetime(df_clean['tgl_pemeriksaan'], errors='coerce')

# Hitung selisih tahun secara akurat (integer bulat)
df_clean['usia'] = ((df_clean['tgl_pemeriksaan'] - df_clean['tgl_lahir']).dt.days / 365.25).fillna(0).astype(int)

# Mengatur posisi kolom usia tepat setelah tgl_lahir
cols = df_clean.columns.tolist()
idx = cols.index('tgl_lahir')
cols.insert(idx + 1, cols.pop(cols.index('usia')))
df_clean = df_clean[cols]

# ==========================================
# LANGKAH 3: HANDLING MISSING VALUES (RATA-RATA PER FITUR)
# ==========================================
# Mengisi nilai kosong dengan rata-rata masing-masing kolom hematologi
for col in cols_numeric:
    rata_rata = df_clean[col].mean()
    df_clean[col] = df_clean[col].fillna(rata_rata)

# Khusus leukosit dan trombosit, kita bulatkan kembali ke Integer setelah imputasi
cols_int = ['leukosit', 'trombosit']
for col in cols_int:
    df_clean[col] = df_clean[col].round().astype('Int64')


# ==========================================
# LANGKAH 4: BINERISASI STATUS EPO
# ==========================================
# 0 = Tidak (Tanpa EPO), 1 = Ya (Dengan EPO)
mapping_epo = {'TIDAK': 0, 'YA': 1}
df_clean['epo'] = df_clean['status_epo'].map(mapping_epo)

# Mengatur posisi status_epo_biner tepat setelah status_epo
cols = df_clean.columns.tolist()
idx_epo = cols.index('status_epo')
cols.insert(idx_epo + 1, cols.pop(cols.index('epo')))
df_clean = df_clean[cols]

# ==========================================
# LANGKAH 5: BINERISASI JENIS KELAMIN
# ==========================================
mapping_jk = {'P': 0, 'L': 1}
df_clean['jk'] = df_clean['jenis_kelamin'].map(mapping_jk)

# Mengatur posisi jenis_kelamin_biner tepat setelah jenis_kelamin
cols = df_clean.columns.tolist()
idx_jk = cols.index('jenis_kelamin')
cols.insert(idx_jk + 1, cols.pop(cols.index('jk')))
df_clean = df_clean[cols]


# ==========================================
# VERIFIKASI HASIL
# ==========================================
print(f"Jumlah Missing Value:\n{df_clean[cols_numeric].isnull().sum()}")
print("-" * 30)
display(df_clean[['tgl_pemeriksaan', 'usia', 'hemoglobin', 'leukosit', 'trombosit', 'epo', 'jk']].head(10))

Jumlah Missing Value:
eritrosit     0
hematokrit    0
MCHC          0
MCH           0
MCV           0
hemoglobin    0
leukosit      0
trombosit     0
dtype: int64
------------------------------


,tgl_pemeriksaan,usia,hemoglobin,leukosit,trombosit,epo,jk
0,2025-03-01,59,8.6,7830,309000,0,0
1,2025-04-02,59,8.1,7150,262000,0,0
2,2025-05-03,59,8.3,8070,297000,0,0
3,2025-06-04,59,7.9,7960,291000,0,0
4,2025-07-02,59,7.5,7860,307000,0,0
5,2025-08-02,60,7.9,9600,315000,0,0
6,2025-08-23,60,7.9,9510,352000,0,0
7,2025-09-03,60,9.5,8710,343000,0,0
8,2025-10-01,60,9.1,9860,269000,0,0
9,2025-11-01,60,8.7,10140,285000,0,0


In [115]:
df_clean.info()

print("📋 Cuplikan 12 Baris Pertama Dataset Raw:")
display(df.head(12))

print("📋 Cuplikan 12 Baris Pertama Dataset Clean:")
display(df_clean.head(12))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 829 entries, 0 to 828
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_pasien        829 non-null    int64         
 1   jenis_kelamin    829 non-null    object        
 2   jk               829 non-null    int64         
 3   tgl_lahir        829 non-null    datetime64[ns]
 4   usia             829 non-null    int64         
 5   tgl_pemeriksaan  829 non-null    datetime64[ns]
 6   eritrosit        829 non-null    float64       
 7   hematokrit       829 non-null    float64       
 8   MCHC             829 non-null    float64       
 9   MCH              829 non-null    float64       
 10  MCV              829 non-null    float64       
 11  hemoglobin       829 non-null    float64       
 12  leukosit         829 non-null    Int64         
 13  trombosit        829 non-null    Int64         
 14  status_epo       829 non-null    object   

,id_pasien,jenis_kelamin,tgl_lahir,tgl_pemeriksaan,eritrosit,hematokrit,MCHC,MCH,MCV,hemoglobin,leukosit,trombosit,status_epo
0,1,P,1965-07-04,2025-03-01,2.8,25.4,33.9,30.2,89.4,8.6,7830,309000,TIDAK
1,1,P,1965-07-04,2025-04-02,2.7,24.3,33.3,29.7,88.9,8.1,7150,262000,TIDAK
2,1,P,1965-07-04,2025-05-03,2.8,24.5,33.9,29.8,88.4,8.3,8070,297000,TIDAK
3,1,P,1965-07-04,2025-06-04,2.7,24.0,32.9,29.2,88.6,7.9,7960,291000,TIDAK
4,1,P,1965-07-04,2025-07-02,2.5,22.5,33.3,29.7,89.5,7.5,7860,307000,TIDAK
5,1,P,1965-07-04,2025-08-02,2.7,24.2,32.6,29.5,89.7,7.9,9600,315000,TIDAK
6,1,P,1965-07-04,2025-08-23,2.9,25.2,32.1,28.6,88.4,7.9,9510,352000,TIDAK
7,1,P,1965-07-04,2025-09-03,3.3,29.3,32.4,28.4,87.6,9.5,8710,343000,TIDAK
8,1,P,1965-07-04,2025-10-01,3.2,28.0,32.9,28.6,87.2,9.1,9860,269000,TIDAK
9,1,P,1965-07-04,2025-11-01,3.1,27.0,32.2,28.3,88.1,8.7,10140,285000,TIDAK


📋 Cuplikan 12 Baris Pertama Dataset Clean:


,id_pasien,jenis_kelamin,jk,tgl_lahir,usia,tgl_pemeriksaan,eritrosit,hematokrit,MCHC,MCH,MCV,hemoglobin,leukosit,trombosit,status_epo,epo
0,1,P,0,1965-07-04,59,2025-03-01,2.8,25.4,33.900000,30.200000,89.400000,8.6,7830,309000,TIDAK,0
1,1,P,0,1965-07-04,59,2025-04-02,2.7,24.3,33.300000,29.700000,88.900000,8.1,7150,262000,TIDAK,0
2,1,P,0,1965-07-04,59,2025-05-03,2.8,24.5,33.900000,29.800000,88.400000,8.3,8070,297000,TIDAK,0
3,1,P,0,1965-07-04,59,2025-06-04,2.7,24.0,32.900000,29.200000,88.600000,7.9,7960,291000,TIDAK,0
4,1,P,0,1965-07-04,59,2025-07-02,2.5,22.5,33.300000,29.700000,89.500000,7.5,7860,307000,TIDAK,0
5,1,P,0,1965-07-04,60,2025-08-02,2.7,24.2,32.600000,29.500000,89.700000,7.9,9600,315000,TIDAK,0
6,1,P,0,1965-07-04,60,2025-08-23,2.9,25.2,32.100000,28.600000,88.400000,7.9,9510,352000,TIDAK,0
7,1,P,0,1965-07-04,60,2025-09-03,3.3,29.3,32.400000,28.400000,87.600000,9.5,8710,343000,TIDAK,0
8,1,P,0,1965-07-04,60,2025-10-01,3.2,28.0,32.900000,28.600000,87.200000,9.1,9860,269000,TIDAK,0
9,1,P,0,1965-07-04,60,2025-11-01,3.1,27.0,32.200000,28.300000,88.100000,8.7,10140,285000,TIDAK,0


In [116]:
df_clean = df_clean.drop(columns=['tgl_lahir', 'status_epo', 'jenis_kelamin'])
print(f"Kolom saat ini: {df_clean.columns.tolist()}")

Kolom saat ini: ['id_pasien', 'jk', 'usia', 'tgl_pemeriksaan', 'eritrosit', 'hematokrit', 'MCHC', 'MCH', 'MCV', 'hemoglobin', 'leukosit', 'trombosit', 'epo']


In [117]:
# 1. Pastikan kolom tgl_pemeriksaan sudah bertipe datetime
df_clean['tgl_pemeriksaan'] = pd.to_datetime(df_clean['tgl_pemeriksaan'])

# 2. Daftar kolom yang akan dirata-ratakan (termasuk usia dan status_epo_biner)
# Kita sertakan usia agar tetap ada di dataset hasil akhir
cols_to_mean = ['usia', 'jk', 'eritrosit', 'hematokrit', 'MCHC', 'MCH', 'MCV', 
                'hemoglobin', 'leukosit', 'trombosit', 'epo']

# 3. Proses Aggregation: Group by id_pasien dan Resample per Bulan (Month Start / MS)
df_clean = (df_clean.groupby('id_pasien')
            .resample('MS', on='tgl_pemeriksaan')[cols_to_mean]
            .mean()
            .dropna() # Menghapus bulan yang memang tidak ada pemeriksaannya
            .reset_index())

# 4. Kembalikan kolom leukosit, trombosit, dan usia ke tipe Integer (karena rata-rata menghasilkan float)
cols_to_int = ['usia', 'jk', 'leukosit', 'trombosit', 'epo']
for col in cols_to_int:
    df_clean[col] = df_clean[col].round().astype('Int64')

# 5. Verifikasi hasil untuk melihat perubahan baris
print(f"Data berhasil diringkas per bulan.")
display(df_clean.head(10))

Data berhasil diringkas per bulan.


,id_pasien,tgl_pemeriksaan,usia,jk,eritrosit,hematokrit,MCHC,MCH,MCV,hemoglobin,leukosit,trombosit,epo
0,1,2025-03-01,59,0,2.8,25.4,33.90,30.20,89.40,8.6,7830,309000,0
1,1,2025-04-01,59,0,2.7,24.3,33.30,29.70,88.90,8.1,7150,262000,0
2,1,2025-05-01,59,0,2.8,24.5,33.90,29.80,88.40,8.3,8070,297000,0
3,1,2025-06-01,59,0,2.7,24.0,32.90,29.20,88.60,7.9,7960,291000,0
4,1,2025-07-01,59,0,2.5,22.5,33.30,29.70,89.50,7.5,7860,307000,0
5,1,2025-08-01,60,0,2.8,24.7,32.35,29.05,89.05,7.9,9555,333500,0
6,1,2025-09-01,60,0,3.3,29.3,32.40,28.40,87.60,9.5,8710,343000,0
7,1,2025-10-01,60,0,3.2,28.0,32.90,28.60,87.20,9.1,9860,269000,0
8,1,2025-11-01,60,0,3.1,27.0,32.20,28.30,88.10,8.7,10140,285000,0
9,1,2025-12-01,60,0,3.0,26.3,32.70,28.80,87.70,8.6,11750,400000,0


In [118]:
df_clean.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_pasien        628 non-null    int64         
 1   tgl_pemeriksaan  628 non-null    datetime64[ns]
 2   usia             628 non-null    Int64         
 3   jk               628 non-null    Int64         
 4   eritrosit        628 non-null    float64       
 5   hematokrit       628 non-null    float64       
 6   MCHC             628 non-null    float64       
 7   MCH              628 non-null    float64       
 8   MCV              628 non-null    float64       
 9   hemoglobin       628 non-null    float64       
 10  leukosit         628 non-null    Int64         
 11  trombosit        628 non-null    Int64         
 12  epo              628 non-null    Int64         
dtypes: Int64(5), datetime64[ns](1), float64(6), int64(1)
memory usage: 67.0 KB


In [119]:
# SIMPAN DATA CLEAN erm_hd_clean.xlsx
path_tujuan = r'E:\airta drafts\PREDIKSI KADAR HB\data\processed\erm_hd_clean.xlsx'

df_clean.to_excel(path_tujuan, index=False)
print(f"Data berhasil disimpan ke: {path_tujuan}")

Data berhasil disimpan ke: E:\airta drafts\PREDIKSI KADAR HB\data\processed\erm_hd_clean.xlsx


In [124]:
import pandas as pd
import os

# 1. Path tujuan Anda
path_tujuan = r'E:\airta drafts\PREDIKSI KADAR HB\data\processed\erm_hd_clean.xlsx'

# 2. Pastikan folder tersedia
folder_tujuan = os.path.dirname(path_tujuan)
if not os.path.exists(folder_tujuan):
    os.makedirs(folder_tujuan)

# 3. Simpan menggunakan Engine XlsxWriter (Wajib untuk mengatur format tampilan)
with pd.ExcelWriter(path_tujuan, engine='xlsxwriter') as writer:
    # Tulis dataframe ke excel
    df_clean.to_excel(writer, index=False, sheet_name='Sheet1')
    
    # Ambil workbook dan worksheet untuk diatur formatnya
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    
    # Definisikan format tanggal (YYYY-MM-DD)
    format_tgl = workbook.add_format({'num_format': 'yyyy-mm-dd'})
    
    # Cari di mana posisi kolom 'tgl_pemeriksaan'
    idx_tgl = df_clean.columns.get_loc('tgl_pemeriksaan')
    
    # Terapkan format ke kolom tersebut agar VS Code membacanya sebagai tanggal
    # set_column(indeks_awal, indeks_akhir, lebar_kolom, format)
    worksheet.set_column(idx_tgl, idx_tgl, 18, format_tgl)

print(f"✅ Selesai! Sekarang buka file di VS Code, kolom tgl_pemeriksaan sudah normal.")

✅ Selesai! Sekarang buka file di VS Code, kolom tgl_pemeriksaan sudah normal.


In [ ]:
# Memilih kolom hematologi utama sesuai rencana Anda
cols_hematologi = ['eritrosit', 'hematokrit', 'MCHC', 'MCH', 'MCV', 'hemoglobin', 'leukosit', 'trombosit']

# Menampilkan statistik deskriptif
print("Statistik Deskriptif Variabel Hematologi:")
display(df_clean[cols_hematologi].describe())

In [ ]:
# Membuat figure untuk subplots
plt.figure(figsize=(15, 12))

for i, col in enumerate(cols_hematologi):
    # Histogram (Baris 1 & 2)
    plt.subplot(4, 4, i+1)
    sns.histplot(df_clean[col], kde=True, color='skyblue') # Menggunakan df_clean
    plt.title(f'Distribusi {col}')
    plt.xlabel('')

    # Boxplot (Baris 3 & 4)
    plt.subplot(4, 4, i+9) 
    sns.boxplot(x=df_clean[col], color='lightcoral') # Menggunakan df_clean
    plt.title(f'Outlier {col}')
    plt.xlabel('')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

# Menghitung korelasi (Spearman)
# Spearman digunakan untuk menangkap hubungan non-linear yang monotonik
correlation_matrix = df_clean[cols_hematologi].corr(method='spearman')

# Visualisasi dengan Heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Heatmap Korelasi Spearman Parameter Hematologi terhadap Hb")
plt.show()

In [ ]:
# Pastikan kolom tanggal adalah datetime
df_clean['tgl_pemeriksaan'] = pd.to_datetime(df_clean['tgl_pemeriksaan'])

# Contoh visualisasi tren Hb untuk 3-5 pasien pertama
plt.figure(figsize=(20, 10))
top_patients = df_clean['id_pasien'].unique()[:5] # Ambil 5 ID pasien pertama 
for patient in top_patients:
    patient_data = df[df['id_pasien'] == patient].sort_values('tgl_pemeriksaan')
    plt.plot(patient_data['tgl_pemeriksaan'], patient_data['hemoglobin'], marker='o', label=f'Pasien {patient}')

plt.title("Tren Fluktuasi Kadar Hb Pasien (Bulan ke Bulan)")
plt.xlabel("Bulan/Tahun")
plt.ylabel("Kadar Hb (g/dL)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(15, 7))

# Ambil 5 ID pasien pertama
top_patients = df_clean['id_pasien'].unique()[:5]

for i, patient in enumerate(top_patients):
    # Filter data per pasien
    patient_data = df_clean[df_clean['id_pasien'] == patient].copy()
    
    # Set tanggal sebagai index untuk proses resampling
    patient_data.set_index('tgl_pemeriksaan', inplace=True)
    
    # Resample per bulan ('MS' = Month Start), ambil rata-rata Hb jika ada >1 pemeriksaan
    monthly_data = patient_data['hemoglobin'].resample('MS').mean().dropna()
    
    plt.plot(monthly_data.index, monthly_data.values, marker='o', label=f'Pasien {i+1}')

plt.title("Tren Rata-Rata Kadar Hb Pasien per Bulan (Data Resampled)")
plt.xlabel("Tahun-Bulan")
plt.ylabel("Kadar Hb (g/dL)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import pandas as pd

# 1. Menyiapkan data 'Before' (Data asli dengan 2 pemeriksaan)
# Pastikan tgl_pemeriksaan sudah datetime
df['tgl_pemeriksaan'] = pd.to_datetime(df['tgl_pemeriksaan'])
tabel_before = df[(df['id_pasien'] == 2) & 
                  (df['tgl_pemeriksaan'].dt.month == 6) & 
                  (df['tgl_pemeriksaan'].dt.year == 2025)].sort_values('tgl_pemeriksaan')

# 2. Menjalankan ulang proses perataan (hanya untuk pengecekan ini)
# Jika df_clean Anda belum berubah, jalankan logika ini
tabel_after = tabel_before.set_index('tgl_pemeriksaan').resample('MS').mean().reset_index()

print("=== [BEFORE] TABEL DATA ASLI (PASIEN ID 2 - JUNI) ===")
display(tabel_before[['id_pasien', 'tgl_pemeriksaan', 'hemoglobin']])

print("\n=== [AFTER] TABEL DATA TERPROSES (HASIL RATA-RATA BULANAN) ===")
display(tabel_after[['id_pasien', 'tgl_pemeriksaan', 'hemoglobin']])

# Narasi untuk Skripsi
print(f"\nAnalisis: Data mentah menunjukkan fluktuasi Hb (7.5 ke 8.3) dalam waktu 4 hari.")
print(f"Hasil rata-rata {tabel_after['hemoglobin'].values[0]} digunakan sebagai nilai baseline bulan Juni.")